In [ ]:
"""
This notebook upsamples motor features in time domain to match the time points of 
BISC recording, discarding segments where BISC recording does not exist). Then, it
normalizes the remaining features.
"""

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys

import numpy as np

from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_motor_global import *

from scipy.interpolate import interp1d

ROOT_SAVE_DIR = f'{MODEL_INPUT_DIR}/motion'

In [ ]:
# initialize
wrist_pos_xs = []
wrist_pos_ys = []
wrist_pos_zs = []
wrist_vel_xs = []
wrist_vel_ys = []
wrist_vel_zs = []

for session in GOOD_SESSIONS:
    keys = [key for key in SESSION_KEYS if key.startswith(f'{session:003}')]

    for key in keys:
        load_dir = f'{REC_DIR}/6_truncate/{key}'
        if not os.path.exists(load_dir):
            continue
        print(f'processing {key}...')

        """ save directory """
        save_data_dir = f'{ROOT_SAVE_DIR}/{key}'
        if not os.path.exists(save_data_dir):
            os.makedirs(save_data_dir)

        """ load recording data """
        spect_t     = np.load(f'{load_dir}/spect_t_{key}.npy')

        """ load motion data """
        motion_dir = f'{MOTION_DIR}/{session:03}'
        motion_t  = np.load(f'{motion_dir}/pos_t_session_{key}.npy')
        wrist_pos_x   = np.load(f'{motion_dir}/wrist_pos_x_session_{key}.npy')
        wrist_pos_y   = np.load(f'{motion_dir}/wrist_pos_y_session_{key}.npy')
        wrist_pos_z   = np.load(f'{motion_dir}/wrist_pos_z_session_{key}.npy')

        wrist_vel_x   = np.load(f'{motion_dir}/wrist_vel_x_session_{key}.npy')
        wrist_vel_y   = np.load(f'{motion_dir}/wrist_vel_y_session_{key}.npy')
        wrist_vel_z   = np.load(f'{motion_dir}/wrist_vel_z_session_{key}.npy')

        wrist_vel_x = np.append(wrist_vel_x, wrist_vel_x[-1])
        wrist_vel_y = np.append(wrist_vel_y, wrist_vel_y[-1])
        wrist_vel_z = np.append(wrist_vel_z, wrist_vel_z[-1])

        # define interp1d funcs
        f_px = interp1d(motion_t, wrist_pos_x)
        f_py = interp1d(motion_t, wrist_pos_y)
        f_pz = interp1d(motion_t, wrist_pos_z)

        f_vx = interp1d(motion_t, wrist_vel_x)
        f_vy = interp1d(motion_t, wrist_vel_y)
        f_vz = interp1d(motion_t, wrist_vel_z)

        """ model time scale """
        idx0 = np.where(spect_t >= spect_t[0] - FULL_TAU_START)[0][0]
        idx1 = np.where(spect_t <= spect_t[-1] - FULL_TAU_END)[0][-1] + 1
        model_t = spect_t[idx0:idx1]

        """ truncate and upsample motion data """
        # interpolate onto BISC recording time points
        interp_wrist_pos_x = f_px(model_t)
        interp_wrist_pos_y = f_py(model_t)
        interp_wrist_pos_z = f_pz(model_t)

        interp_wrist_vel_x = f_vx(model_t)
        interp_wrist_vel_y = f_vy(model_t)
        interp_wrist_vel_z = f_vz(model_t)

        """ save data """
        # truncated and interpolated motion data
        np.save(f'{save_data_dir}/wrist_pos_x_{key}.npy', interp_wrist_pos_x)
        np.save(f'{save_data_dir}/wrist_pos_y_{key}.npy', interp_wrist_pos_y)
        np.save(f'{save_data_dir}/wrist_pos_z_{key}.npy', interp_wrist_pos_z)

        np.save(f'{save_data_dir}/wrist_vel_x_{key}.npy', interp_wrist_vel_x)
        np.save(f'{save_data_dir}/wrist_vel_y_{key}.npy', interp_wrist_vel_y)
        np.save(f'{save_data_dir}/wrist_vel_z_{key}.npy', interp_wrist_vel_z)

        np.save(f'{save_data_dir}/model_t_{key}.npy', model_t)

        """ append """
        wrist_pos_xs.append(interp_wrist_pos_x)
        wrist_pos_ys.append(interp_wrist_pos_y)
        wrist_pos_zs.append(interp_wrist_pos_z)
        
        wrist_vel_xs.append(interp_wrist_vel_x)
        wrist_vel_ys.append(interp_wrist_vel_y)
        wrist_vel_zs.append(interp_wrist_vel_z)

In [ ]:
# convert to array
wrist_pos_x_sigma = np.std(np.concatenate(wrist_pos_xs, axis=-1))
wrist_pos_y_sigma = np.std(np.concatenate(wrist_pos_ys, axis=-1))
wrist_pos_z_sigma = np.std(np.concatenate(wrist_pos_zs, axis=-1))
wrist_vel_x_sigma = np.std(np.concatenate(wrist_vel_xs, axis=-1))
wrist_vel_y_sigma = np.std(np.concatenate(wrist_vel_ys, axis=-1))
wrist_vel_z_sigma = np.std(np.concatenate(wrist_vel_zs, axis=-1))

In [ ]:
""" normalize motion data """
for key in SESSION_KEYS:
    load_dir = f'{ROOT_SAVE_DIR}/{key}'
    if not os.path.exists(load_dir):
        continue
    print(f'processing {key}...')

    """ load back unnormalized motion data """
    wrist_pos_x   = np.load(f'{load_dir}/wrist_pos_x_{key}.npy')
    wrist_pos_y   = np.load(f'{load_dir}/wrist_pos_y_{key}.npy')
    wrist_pos_z   = np.load(f'{load_dir}/wrist_pos_z_{key}.npy')
    wrist_vel_x   = np.load(f'{load_dir}/wrist_vel_x_{key}.npy')
    wrist_vel_y   = np.load(f'{load_dir}/wrist_vel_y_{key}.npy')
    wrist_vel_z   = np.load(f'{load_dir}/wrist_vel_z_{key}.npy')

    """ save normalized version """
    save_dir = load_dir
    np.save(f'{save_dir}/norm_wrist_pos_x_{key}.npy', wrist_pos_x/wrist_pos_x_sigma)
    np.save(f'{save_dir}/norm_wrist_pos_y_{key}.npy', wrist_pos_y/wrist_pos_y_sigma)
    np.save(f'{save_dir}/norm_wrist_pos_z_{key}.npy', wrist_pos_z/wrist_pos_z_sigma)

    np.save(f'{save_dir}/norm_wrist_vel_x_{key}.npy', wrist_vel_x/wrist_vel_x_sigma)
    np.save(f'{save_dir}/norm_wrist_vel_y_{key}.npy', wrist_vel_y/wrist_vel_y_sigma)
    np.save(f'{save_dir}/norm_wrist_vel_z_{key}.npy', wrist_vel_z/wrist_vel_z_sigma)

In [ ]:
dims = ['x', 'y', 'z']

for key in SESSION_KEYS:
    for dim in dims:
        fn0 = f'norm_wrist_{dim}_{key}.npy'
        fn1 = f'norm_wrist_pos_{dim}_{key}.npy'

        # fn0 = f'wrist_{dim}_{key}.npy'
        # fn1 = f'wrist_pos_{dim}_{key}.npy'

        # fn0 = f'wrist_vel_{dim}_{key}.npy'
        # fn1 = f'wrist_vel_{dim}_{key}.npy'

        # fn0 = f'norm_wrist_vel_{dim}_{key}.npy'
        # fn1 = f'norm_wrist_vel_{dim}_{key}.npy'

        fn0 = f'scalo_t_{key}.npy'
        fn1 = f'model_t_{key}.npy'

        dir0 = f'./combined_preprocessed_v3/1_align_and_chop_again/{key}'
        dir1 = f'./model_input/motion/{key}'

        X0 = np.load(f'{dir0}/{fn0}')
        X1 = np.load(f'{dir1}/{fn1}')
        assert np.allclose(X0, X1, equal_nan=True)